In [29]:
import requests
import json
import time
import polars as pl   
import base64
import os
import msgpack
from tkinter import Tcl

In [30]:
# Mapping the 3 different API call page types to variables referencing relevant file directories.

base_directory =  "data/"

tmp_dir_staking_transactions = base_directory + "tmp_staking_transactions/"
tmp_dir_staked_accounts = base_directory + "tmp_staked_accounts/"
tmp_dir_sensor_transactions = base_directory + "tmp_sensor_transactions/"

staking_transactions = base_directory + "staking_transactions/"
staked_accounts = base_directory + "staked_accounts/"
sensor_transactions = base_directory + "sensor_transactions/"

staking_transactions_cleaned = base_directory + "staking_transactions_cleaned/"
staked_accounts_cleaned  = base_directory + "staked_accounts_cleaned/"
sensor_transactions_cleaned  = base_directory + "sensor_transactions_cleaned/"

staking_page = "staking_page_"
sensor_page = "transaction_page_"
account_page = "account_page_"

# PlanetWatch token blockchain ID

asset_id = "27165954"

# Loading date from most recent run to be imported into API call

previous_time = pl.read_json(staking_transactions_cleaned + "cleaned_staked_pages.json").select("stakeTransactionDate").to_dict(as_series=False)["stakeTransactionDate"][0]
previous_time = str(previous_time)

after_time = previous_time

# Base JSON scraping function and natural sorted directory func

def json_scraper(url, file_name):
    response = requests.request("GET", url)
    json_data = response.json()
    with open(file_name, "w", encoding= "utf-8") as json_file:
        json.dump(json_data, json_file, ensure_ascii=False, indent=4)

def nat_sort_dir(parent_directory):
    directory = os.listdir(parent_directory)
    list_of_files = Tcl().call('lsort', '-dict', directory)
    return list_of_files

In [31]:
def staking_transaction_scraper_loop(tmp_dir, after_time, page_type):
    pages_remaining = True
    page = 1
    current_json = tmp_dir + page_type + "1.json"
    attempts = 0
    sleep = 5
    
    json_scraper("https://mainnet-idx.algonode.cloud/v2/assets/" + asset_id + "/transactions?after-time=" + after_time + "&address=4X65WSIAVN5KTOURSM77PJXFLJ7HDLTEQ4A26BQLHSXQ7TOSY6ML3BJWKU&address-role=sender", current_json)
    
    
    while pages_remaining:
            file = open(current_json)
            data = json.load(file)
            
            data
            base_url = "https://mainnet-idx.algonode.cloud/v2/assets/" + asset_id + "/transactions?after-time=" + after_time + "&address=4X65WSIAVN5KTOURSM77PJXFLJ7HDLTEQ4A26BQLHSXQ7TOSY6ML3BJWKU&address-role=sender"

            try:
                new_url = base_url[:base_url.index("?")] + "?next=" + data["next-token"] + "&after-time=" + after_time + "&address=4X65WSIAVN5KTOURSM77PJXFLJ7HDLTEQ4A26BQLHSXQ7TOSY6ML3BJWKU&address-role=sender"
                page += 1
                file.close()
                current_json = tmp_dir + page_type + str(page) + ".json"
                attempts = 0
                sleep = 5
            except:
                print(data)
                if attempts < 4:
                    time.sleep(sleep)
                    sleep += 5
                    attempts += 1
                    file = open(tmp_dir + page_type + str(page - 1) + ".json")
                    data = json.load(file)
                    new_url = base_url[:base_url.index("?")] + "?next=" + data["next-token"] + "&after-time=" + after_time + "&address=4X65WSIAVN5KTOURSM77PJXFLJ7HDLTEQ4A26BQLHSXQ7TOSY6ML3BJWKU&address-role=sender"
                    file.close()
                    current_json = tmp_dir + page_type + str(page) + ".json"
                else:
                     break
            
            json_scraper(new_url, current_json)


In [32]:
# Decoder function used for decoding the Base64 string baked into the note column of staking transactions

def msgpack_decoder(b64_string):
    decoded = msgpack.unpackb(base64.b64decode(b64_string))
    return decoded

def staking_transaction_cleaning(file):
    staked = pl.read_json(file)
    staked = staked.drop("current-round", "next-token")
    staked = staked.explode("transactions")
    staked = staked.unnest("transactions")
    staked = staked.drop("closing-amount", "close-rewards", "fee", "first-valid", "confirmed-round", "genesis-hash", "genesis-id", "intra-round-offset", "last-valid", "receiver-rewards", "round-time", "sender", "sender-rewards", "signature", "tx-type")
    staked = staked.unnest("asset-transfer-transaction")
    staked = staked.drop("asset-id", "close-amount", "amount")

    vals = [row["note"] for row in staked.iter_rows(named=True)]
    for i in vals:
        vals[vals.index(i)] = msgpack_decoder(i)
    for i in vals:
        if type(i["hodler_program"]) != float:
            i["hodler_program"] = float(i["hodler_program"])
        if type(i["total_from_the_beginning"]) != float:
            i["total_from_the_beginning"] = float(i["total_from_the_beginning"])

    staked = staked.with_columns(pl.Series(name="decoded_notes", values=vals))
    staked = staked.unnest("decoded_notes")
    staked_clean = staked.drop("note", "hodler_qualifying")
    return staked_clean



In [33]:
def staked_account_scraper_loop(staked_df, page_type):
    unique_accounts = staked_df.unique("deviceId", maintain_order=True)
    unique_accounts = unique_accounts.unique("accountAddress", maintain_order=True)
    unique_accounts = [row["accountAddress"] for row in unique_accounts.iter_rows(named=True)]
    page = 1

    for i in unique_accounts:
        json_scraper("https://mainnet-idx.algonode.cloud/v2/accounts/" + i + "/assets?asset-id=" + asset_id, tmp_dir_staked_accounts + page_type + str(page) + ".json")
        page += 1
    

In [34]:
def staked_account_cleaning(file):
    staked = pl.read_json(file)
    try:
        staked = staked.drop("next-token", "current-round")
        staked = staked.explode("assets")
        staked = staked.unnest("assets")
        staked = staked.drop("asset-id", "is-frozen", "opted-in-at-round")
        staked = staked.replace("amount", pl.Series([row["amount"] / 1000000 for row in staked.iter_rows(named=True)]))
        staked_acount_cleaned = staked.rename({"amount":"balance"})
        return staked_acount_cleaned
    except:
        staked = staked.replace("current-round", pl.Series([0], dtype=pl.Float64))
        staked_acount_cleaned = staked.rename({"current-round":"balance"})
        return staked_acount_cleaned


In [35]:
def sensor_transaction_scraper_loop(tmp_dir, after_time, page_type):
    pages_remaining = True
    page = 1
    current_json = tmp_dir + page_type + ".json"
    attempts = 0
    sleep = 5
    
    json_scraper("https://mainnet-idx.algonode.cloud/v2/transactions?tx-type=axfer&asset-id=" + asset_id + "&after-time=" + after_time + "&currency-greater-than=0&address=ZW3ISEHZUHPO7OZGMKLKIIMKVICOUDRCERI454I3DB2BH52HGLSO67W754&address-role=sender", current_json)
    
    
    while pages_remaining:
            file = open(current_json)
            data = json.load(file)
            
            data
            base_url = "https://mainnet-idx.algonode.cloud/v2/transactions?tx-type=axfer&asset-id=" + asset_id + "&after-time=" + after_time + "&currency-greater-than=0&address=ZW3ISEHZUHPO7OZGMKLKIIMKVICOUDRCERI454I3DB2BH52HGLSO67W754&address-role=sender"

            try:
                new_url = base_url[:base_url.index("?")] + "?next=" + data["next-token"] + "&tx-type=axfer&asset-id=" + asset_id + "&after-time=" + after_time + "&currency-greater-than=0&address=ZW3ISEHZUHPO7OZGMKLKIIMKVICOUDRCERI454I3DB2BH52HGLSO67W754&address-role=sender"
                page += 1
                file.close()
                current_json = tmp_dir + page_type + str(page) + ".json"
                attempts = 0
                sleep = 5
            except:
                print(data)
                if attempts < 4:
                    time.sleep(sleep)
                    sleep += 5
                    attempts += 1
                    file = open(tmp_dir + page_type + str(page - 1) + ".json")
                    data = json.load(file)
                    new_url = base_url[:base_url.index("?")] + "?next=" + data["next-token"] + "&tx-type=axfer&asset-id=" + asset_id + "&after-time=" + after_time + "&currency-greater-than=0&address=ZW3ISEHZUHPO7OZGMKLKIIMKVICOUDRCERI454I3DB2BH52HGLSO67W754&address-role=sender"
                    file.close()
                    current_json = tmp_dir + page_type + str(page) + ".json"
                else:
                     break
            
            json_scraper(new_url, current_json)

In [36]:
# Decoder function used for decoding the Base64 string baked into the note column of sensor transactions

def decoder(b64_string):
    decoded = json.loads(base64.b64decode(b64_string).decode("utf-8"))
    return decoded

def sensor_transaction_cleaning(file):
    transact = pl.read_json(file)
    transact = transact.drop("current-round", "next-token")
    transact = transact.explode("transactions")
    transact = transact.unnest("transactions")
    transact = transact.drop("closing-amount", "close-rewards", "fee", "first-valid", "confirmed-round", "genesis-hash", "genesis-id", "group", "intra-round-offset", "last-valid", "receiver-rewards", "round-time", "sender", "sender-rewards", "signature", "tx-type")
    transact = transact.unnest("asset-transfer-transaction")
    transact = transact.drop("asset-id", "close-amount")

    vals = [row["note"] for row in transact.iter_rows(named=True)]
    for i in vals:
        vals[vals.index(i)] = decoder(i)
    for i in vals:
        if 'scale' in i:
            del i["scale"]   
        if type(i["type"]) != str:
            i["type"] = str(i["type"])
        if 'tier' in i:
            if type(i["tier"]) != str:
                i["tier"] = str(i["tier"])

    try:
        transact = transact.with_columns(pl.Series(name="decoded_notes", values=vals))
        transact = transact.unnest("decoded_notes")
        transact = transact.drop("note")
        transact_cleaned = transact
        return transact_cleaned
    except:
        type_1_2_keys = {'date', 'streams', 'deviceId', 'type', 'status', 'tier', 'qualifying', 'localScore', 'globalScore'}
        type_3_4_keys = {'date', 'streams', 'deviceId', 'type', 'qualifying', 'localScore', 'globalScore'}
        type_3_4_dev = {'amount': [], 'receiver': [], 'id': [], 'note': []}
        type_1_2_dev = {'amount': [], 'receiver': [], 'id': [], 'note': []}
        type_3_4_decoded = []
        type_1_2_decoded = []
        type_3_4_df = pl.DataFrame(type_3_4_dev, schema={'amount': pl.Int64, 'receiver': pl.Utf8, 'id': pl.Utf8, 'note': pl.Utf8})
        type_1_2_df = pl.DataFrame(type_1_2_dev, schema={'amount': pl.Int64, 'receiver': pl.Utf8, 'id': pl.Utf8, 'note': pl.Utf8})

        for i in vals:
            if i.keys() == type_3_4_keys:
                type_3_4_df = pl.concat([type_3_4_df, pl.DataFrame(transact.row(vals.index(i), named=True))])
                type_3_4_decoded.append(i)
            if i.keys() == type_1_2_keys:
                type_1_2_df = pl.concat([type_1_2_df, pl.DataFrame(transact.row(vals.index(i), named=True))])
                type_1_2_decoded.append(i)

        type_3_4_df = type_3_4_df.with_columns(pl.Series(name="decoded_notes", values=type_3_4_decoded))
        type_1_2_df = type_1_2_df.with_columns(pl.Series(name="decoded_notes", values=type_1_2_decoded))
        type_3_4_df = type_3_4_df.unnest("decoded_notes")
        type_1_2_df = type_1_2_df.unnest("decoded_notes")

        combined_type_df = pl.concat([type_3_4_df, type_1_2_df], how="diagonal")
        return combined_type_df



In [37]:
# Not currently in use due to API data limits.

def accounts_by_asset_balance_scraper_loop():
    pages_remaining = True
    page = 1
    current_json = base_directory + "current_accounts_pages/current_account_page_1.json"
    
    json_scraper("https://mainnet-idx.algonode.cloud/v2/assets/27165954/balances?currency-greater-than=6999000000", base_directory + "current_accounts_pages/current_account_page_1.json")
    
    
    while pages_remaining:
            file = open(current_json)
            data = json.load(file)
            
            data
            base_url = "https://mainnet-idx.algonode.cloud/v2/assets/27165954/balances&currency-greater-than=6999000000"
    
            page += 1
            new_url = base_url[:62] + "?next=" + data["next-token"] + "&currency-greater-than=6999000000"
            file.close()
            current_json = base_directory + "current_accounts_pages/current_account_page_" + str(page) + ".json"
    
            json_scraper(new_url, current_json)
            
def accounts_by_asset_balance_cleaning(file):
    accounts_df = pl.read_json(file)
    accounts_df = accounts_df.drop("current-round", "next-token")
    accounts_df = accounts_df.explode("balances")
    accounts_df = accounts_df.unnest("balances")
    accounts_df = accounts_df.rename({"address":"accountAddress"})
    accounts_df = accounts_df.rename({"amount":"balance"})
    try:
        accounts_df = accounts_df.drop("opted-in-at-round", "opted-out-at-round", "is-frozen", "deleted")
    except:
        accounts_df = accounts_df.drop("opted-in-at-round", "is-frozen", "deleted")
    return accounts_df



In [38]:
# Not currently in use due to API data limits.

def account_transactions_scraper_loop(account_string):
    pages_remaining = True
    current_json = base_directory + "accounts_pages/" + account_string + "_page_1.json"
    attempts = 0
    sleep = 5
    current_account = ""

    json_scraper("https://mainnet-idx.algonode.cloud/v2/accounts/" + account_string + "/transactions?asset-id=27165954&currency-greater-than=0", base_directory + "accounts_pages/" + account_string + "_page_1.json")
    
    
    while pages_remaining:
            file = open(current_json)
            data = json.load(file)
            
            data
            base_url = "https://mainnet-idx.algonode.cloud/v2/accounts/" + account_string + "/transactions?asset-id=27165954&currency-greater-than=0"

            try:
                new_url = base_url[:base_url.index("?")] + "?next=" + data["next-token"] + "&asset-id=27165954&currency-greater-than=0"

                if current_account != account_string:
                     current_account = account_string
                     page = 0
                
                page += 1
                file.close()
                current_json = base_directory + "accounts_pages/" + account_string + "_page_" + str(page) + ".json"
                attempts = 0
                sleep = 5
            except:
                print(data)
                if attempts < 4:
                    time.sleep(sleep)
                    sleep += 5
                    attempts += 1
                    file = open(base_directory + "accounts_pages/" + account_string + "_page_" + str(page - 1) + ".json")
                    data = json.load(file)
                    new_url = base_url[:base_url.index("?")] + "?next=" + data["next-token"] + "&asset-id=27165954&currency-greater-than=0"
                    file.close()
                    current_json = base_directory + "accounts_pages/" + account_string + "_page_" + str(page) + ".json"
                else:
                     break
            
            json_scraper(new_url, current_json)

In [39]:
# Function for adjusting page numbers of existing files in raw directory to start at the end of the new files located in temp directory. Relocation of temp directory files to raw file directory

def tmp_dir_cleaning(tmp_dir, uncleaned_dir, page_type):

    last_page = len(tmp_dir)

    for i in nat_sort_dir(uncleaned_dir):
        os.rename(uncleaned_dir + i, tmp_dir + page_type + str(last_page + 1) + ".json")
        last_page += 1
    for i in nat_sort_dir(tmp_dir):
        os.rename(tmp_dir + i, uncleaned_dir + i)

In [12]:
staking_transaction_scraper_loop(tmp_dir_staking_transactions, after_time, staking_page)


{'current-round': 32367481, 'transactions': []}
{'current-round': 32367482, 'transactions': []}
{'current-round': 32367485, 'transactions': []}
{'current-round': 32367490, 'transactions': []}
{'current-round': 32367496, 'transactions': []}


In [13]:
# Cleaning of new JSONs and appending to lists for concatonation, debugging, or deletion

staking_jsons = []

broken_jsons = []

for i in nat_sort_dir(tmp_dir_staking_transactions):
    try:       
        cleaned_file = staking_transaction_cleaning(tmp_dir_staking_transactions + i)
        staking_jsons.append(cleaned_file)
    except:
        print(i)
        broken_jsons.append(i)
        continue
if len(broken_jsons) == 1:
    os.remove(tmp_dir_staking_transactions + broken_jsons[0])

staking_page_55.json


In [14]:
tmp_dir_cleaning(tmp_dir_staking_transactions, staking_transactions, staking_page)

In [27]:
combined_staking_transactions_df = pl.concat(staking_jsons)
combined_staking_transactions_df = combined_staking_transactions_df.rename({"device":"deviceId"})
combined_staking_transactions_df = combined_staking_transactions_df.rename({"id":"stakeTransactionId"})
combined_staking_transactions_df = combined_staking_transactions_df.rename({"date":"stakeTransactionDate"})
combined_staking_transactions_df = combined_staking_transactions_df.with_columns(pl.col("stakeTransactionDate").str.to_date())
combined_staking_transactions_df = combined_staking_transactions_df.rename({"receiver":"accountAddress"})
combined_staking_transactions_df = pl.concat([combined_staking_transactions_df, pl.read_json(staking_transactions_cleaned + "cleaned_staked_pages.json")])
combined_staking_transactions_df = combined_staking_transactions_df.unique("stakeTransactionId", maintain_order=True)
combined_staking_transactions_df.write_json(staking_transactions_cleaned + "cleaned_staked_pages.json")

In [28]:
combined_staking_transactions_df = pl.DataFrame._read_json(staking_transactions_cleaned + "cleaned_staked_pages.json")
combined_staking_transactions_df

accountAddress,stakeTransactionId,category,stakeTransactionDate,deviceId,hodler_program,total_from_the_beginning
str,str,str,date,str,f64,f64
"""GQK6DACLOBA65K…","""HCFTXFQF3QADRN…","""silver""",2023-09-24,"""PW:AB_003082""",26.6,5985.0
"""OOBP5DJRMALYTJ…","""BTRKPJXODUUNRV…","""diamond""",2023-09-24,"""PW:AB_015965""",266.0,60648.0
"""OOBP5DJRMALYTJ…","""R7JMRYTK7SOPFO…","""diamond""",2023-09-24,"""PW:AB_028048""",266.0,56658.0
"""Y4OFYXBM45HOKB…","""AS34SJ3I33X22R…","""silver""",2023-09-24,"""PW:KA_013602""",53.2,8937.6
"""HFPCVESO2XLDIG…","""NLC2BUZF7BPZXW…","""diamond""",2023-09-24,"""PW:AB_014402""",266.0,54530.0
"""BYUO2RMTBOD7DM…","""3GTG4ITDVBK5B3…","""silver""",2023-09-24,"""PW:AQ_007079""",133.0,27265.0
"""BYUO2RMTBOD7DM…","""E2RDPRFX3YMXSX…","""silver""",2023-09-24,"""PW:AQ_007067""",133.0,23408.0
"""BYUO2RMTBOD7DM…","""RQR65XYDRBDCK6…","""silver""",2023-09-24,"""PW:AQ_007062""",133.0,30590.0
"""Y4OFYXBM45HOKB…","""UGTUVA3Z3R6KWW…","""gold""",2023-09-24,"""PW:TU_000198""",266.0,61446.0


In [42]:
staked_account_scraper_loop(combined_staking_transactions_df, account_page)

In [43]:
# Cleaning of new JSONs and appending to lists for concatonation, debugging, or deletion

staked_account_jsons = []

broken_jsons = []

for i in nat_sort_dir(tmp_dir_staked_accounts):
    try:
        cleaned_file = staked_account_cleaning(tmp_dir_staked_accounts + i)
        staked_account_jsons.append(cleaned_file)
    except:
        print(i)
        broken_jsons.append(i)
        continue
if len(broken_jsons) == 1:
    os.remove(tmp_dir_staked_accounts + broken_jsons[0])

In [44]:
tmp_dir_cleaning(tmp_dir_staked_accounts, staked_accounts, account_page)

In [45]:
unique_accounts = combined_staking_transactions_df.unique("deviceId", maintain_order=True)
unique_accounts = unique_accounts.unique("accountAddress", maintain_order=True)

combined_staked_account_df = pl.concat(staked_account_jsons, how="diagonal")
combined_staked_account_df = combined_staked_account_df.drop("opted-out-at-round", "deleted")
combined_staked_account_df = pl.concat([unique_accounts.select("accountAddress"), combined_staked_account_df], how="horizontal")
combined_staked_account_df.write_json(staked_accounts_cleaned +"cleaned_staked_accounts.json")

In [46]:
combined_staked_account_df = pl.DataFrame._read_json(staked_accounts_cleaned +"cleaned_staked_accounts.json")
combined_staked_account_df

accountAddress,balance
str,f64
"""GQK6DACLOBA65K…",31416.140747
"""OOBP5DJRMALYTJ…",141578.554582
"""Y4OFYXBM45HOKB…",91033.423266
"""HFPCVESO2XLDIG…",560010.000079
"""BYUO2RMTBOD7DM…",114270.443206
"""MAXNP7LVUX7MHD…",487241.459024
"""PFTV6AASEGBJYA…",1.0005e6
"""TEFHFBWCW32D7E…",273564.839999
"""U3MYXYWZCYLXG5…",2.94365e6


In [22]:
sensor_transaction_scraper_loop(tmp_dir_sensor_transactions, after_time, sensor_page)

{'message': 'error while searching for transaction: ERROR: canceling statement due to statement timeout (SQLSTATE 57014)'}
{'message': 'error while searching for transaction: ERROR: canceling statement due to statement timeout (SQLSTATE 57014)'}
{'message': 'error while searching for transaction: ERROR: canceling statement due to statement timeout (SQLSTATE 57014)'}
{'current-round': 32367847, 'transactions': []}
{'current-round': 32367850, 'transactions': []}
{'current-round': 32367854, 'transactions': []}
{'current-round': 32367859, 'transactions': []}
{'current-round': 32367866, 'transactions': []}


In [23]:
# Cleaning of new JSONs and appending to lists for concatonation, debugging, or deletion

sensor_transact_jsons = []

broken_jsons = []

for i in nat_sort_dir(tmp_dir_sensor_transactions):
    try:       
        cleaned_file = sensor_transaction_cleaning(tmp_dir_sensor_transactions + i)
        sensor_transact_jsons.append(cleaned_file)
    except:
        print(i)
        broken_jsons.append(i)
        continue
if len(broken_jsons) == 1:
    os.remove(tmp_dir_sensor_transactions + broken_jsons[0])

transaction_page_457.json


In [24]:
tmp_dir_cleaning(tmp_dir_sensor_transactions, sensor_transactions, sensor_page)

In [40]:
combined_sensor_transaction_df = pl.concat(sensor_transact_jsons, how="diagonal")
combined_sensor_transaction_df = combined_sensor_transaction_df.rename({"id":"transactionId"})
combined_sensor_transaction_df = combined_sensor_transaction_df.rename({"receiver":"accountAddress"})
combined_sensor_transaction_df = combined_sensor_transaction_df.rename({"transactionId":"sensorTransactionId"})
combined_sensor_transaction_df = combined_sensor_transaction_df.filter((pl.col("qualifying") != False))
combined_sensor_transaction_df = combined_sensor_transaction_df.drop("qualifying")
#combined_sensor_transaction_df = combined_sensor_transaction_df.filter((pl.col("type") == "1") & (pl.col("streams") >= 48) | (pl.col("type") == "2") & (pl.col("streams") >= 48) | (pl.col("type") == "3") & (pl.col("streams") >= 144) | (pl.col("type") == "4") & (pl.col("streams") >= 463))
combined_sensor_transaction_df = combined_sensor_transaction_df.filter(pl.col("deviceId").is_in([row["deviceId"] for row in combined_staking_transactions_df.iter_rows(named=True)]))
combined_sensor_transaction_df = combined_sensor_transaction_df.with_columns(pl.col("date").str.to_date())
combined_sensor_transaction_df = combined_sensor_transaction_df.replace("amount", pl.Series([row["amount"] / 1000000 for row in combined_sensor_transaction_df.iter_rows(named=True)]))
combined_sensor_transaction_df = pl.concat([combined_sensor_transaction_df, pl.read_json(sensor_transactions_cleaned + "transactions_cleaned.json")])
combined_sensor_transaction_df = combined_sensor_transaction_df.unique("sensorTransactionId", maintain_order=True)
combined_sensor_transaction_df.write_json(sensor_transactions_cleaned + "transactions_cleaned.json")

In [41]:
combined_sensor_transact_df = pl.DataFrame._read_json(sensor_transactions_cleaned + "transactions_cleaned.json")
combined_sensor_transact_df

amount,accountAddress,sensorTransactionId,date,streams,deviceId,type,localScore,globalScore,status,tier
f64,str,str,date,i64,str,str,i64,i64,str,str
11.712,"""BR55CBEWTQXNVY…","""CTIICLAVP2SBVF…",2023-09-24,366,"""PW:AB_001553""","""4""",357,357,null,null
2.928,"""LSAHUHHFNBWVSQ…","""MS3DIR5HORQNPC…",2023-09-24,366,"""PW:AB_001553""","""4""",357,357,null,null
15.904,"""VFDFKPZQS6GP7V…","""OXOY2JCA3KE7JW…",2023-09-24,497,"""PW:AB_003396""","""4""",599,599,null,null
3.976,"""LSAHUHHFNBWVSQ…","""VDAF4SVWHNWJ26…",2023-09-24,497,"""PW:AB_003396""","""4""",599,599,null,null
14.08,"""REEJWQX7T54PJS…","""2HTAVUL7T4N2S2…",2023-09-24,440,"""PW:AB_010622""","""4""",607,607,null,null
3.52,"""LSAHUHHFNBWVSQ…","""NK35OT65EQCF6T…",2023-09-24,440,"""PW:AB_010622""","""4""",607,607,null,null
12.064,"""WYOODKLS2HNPFA…","""5MWJGM3UJBHMYR…",2023-09-24,377,"""PW:AB_004269""","""4""",702,702,null,null
3.016,"""LSAHUHHFNBWVSQ…","""R7QPHHZX4Z7RLL…",2023-09-24,377,"""PW:AB_004269""","""4""",702,702,null,null
15.488,"""3UDL7GFUQ35Z3H…","""PFMJLIKPX5VVBI…",2023-09-24,484,"""PW:AB_027763""","""4""",202,202,null,null
